In [1]:
import numpy as np
import torch
import os
# from cnf.utils.normalize import Normalizer_np

In [4]:
Nt = 40
Ng = 20

# Set filepaths
case = 'NoisyPipe'
training_path = 'training/' + case + '/'
data_path = 'data/' + case + '/data.npy'
coords_path = 'data/' + case + '/coords.npy'
coords_o_path = 'data/' + case + '/coords_o.npy'
sdf_path = 'data/' + case + '/sdf.npy'
sdf_o_path = 'data/' + case + '/sdf_o.npy'

# Get device
if torch.cuda.is_available():
  dev = "cuda"
else:
  dev = "cpu"

device = torch.device(dev)
print(device)

# Load latents
ckpt = torch.load(training_path + 'checkpoint_9999.pt')
latents = ckpt['hidden_states']['latents']
latents = latents.cpu().numpy()

cuda


In [5]:
# Reshape latents
dataset = np.reshape(latents,(Ng,Nt,-1))
print(dataset.shape)

# Train/validation split
np.random.seed(42)
train_indices = np.random.choice(Ng, int(0.8*Ng), replace=False)
train_test_mask = np.zeros(Ng, dtype=bool)
train_test_mask[train_indices] = True
train_data  = dataset[train_test_mask]
valid_data  = dataset[~train_test_mask]
# train_data = np.expand_dims(train_data,axis=1)
# valid_data = np.expand_dims(valid_data,axis=1)
print(train_data.shape,valid_data.shape)

# Get min/max values
# normalizer = Normalizer_np(method='-11',dim=0) 
# normalizer.fit_normalize(train_data) # Determine normalization params only on training set
# train_data = normalizer.normalize(train_data)
# valid_data = normalizer.normalize(valid_data)
# print(np.max(train_data),np.min(train_data),np.max(valid_data),np.min(valid_data))
max_val, min_val = np.max(train_data, keepdims=True), np.min(train_data, keepdims=True) # Taken from train.py
print(max_val,min_val)

# Save
save_path = '/home/dana/Documents/CoNFiLD/CoNFiLD/UnconditionalDiffusionTraining_and_Generation/data/NoisyPipe/'
os.makedirs(save_path, exist_ok=True)
np.save(save_path + 'train_data.npy', train_data)
np.save(save_path + 'valid_data.npy', valid_data)
np.save(save_path + 'max.npy', max_val)
np.save(save_path + 'min.npy', min_val)

(20, 40, 40)
(16, 40, 40) (4, 40, 40)
[[[0.1209851]]] [[[-0.11907468]]]
